In [ ]:
from geonamescache import GeonamesCache
from unidecode import unidecode
import pandas as pd
import re
import numpy as np

gc = GeonamesCache()

headline = ''
country = np.nan
city = np.nan
columns = ['headline','countries','cities']
frame = pd.DataFrame(columns=columns)

city_names = gc.get_cities()
country_names = gc.get_countries_by_names()

list_of_city_names = []
list_of_country_names = []

#list of cities
for city_name in city_names.values():
    list_of_city_names.append(city_name['name'])
    
#list of countries 
for country_name in country_names:
    list_of_country_names.append(country_name)


list_of_name_list = []

#reading headlines.txt file
with open('data/headlines.txt','r') as file:
    for line in file.readlines():
        list_of_name_list.append(unidecode(line).splitlines())
        
flat_list = []

#initiailizing dataframe and normalizing
for sublist in list_of_name_list:
    for item in sublist:
        flat_list.append(item)
        obj = {'headline' : item, 'countries': np.nan, 'cities': np.nan}
        frame = frame.append(obj,ignore_index=True)
        
#extract cities and update cities to dataframe
for idx,headline_name in enumerate(flat_list):
    longest = 0
    for city_name in city_names.values():
        #need to use unidecode to normalize city name and to have no repeat in city output 
        match_city = re.search(r"\b{}\b".format(unidecode(city_name['name'])),unidecode(headline_name))
        if match_city is not None:
            #check len of city found before saving since it
            #e.g. for a line - San Juan reports 1st U.S. Zika-related death amid outbreak
            #possible output for city are - San,and then San Juan
            #we should deal the above exception by their length
            if len(unidecode(city_name['name'])) > longest:
                i,j = match_city.start(),match_city.end()
                city = headline_name[i: j]
                longest = len(city)
                frame.at[idx,'cities'] = city


#extract countries and update countries to dataframe
for idx,headline_name in enumerate(flat_list):
    for country_name in list_of_country_names:
        match_country = re.search(r"\b{}\b".format(unidecode(country_name)), unidecode(headline_name))
        if match_country is not None:
            start,end = match_country.start(), match_country.end()
            country = headline_name[start: end]
            frame.at[idx,'countries'] = country

print(frame.to_string(index=False))